### WebCrawling
- 웹페이지의 종류
    - 정적페이지 : 페이지의 데이터가 변경될때 URL이 변경(다른 페이지 보기 클릭 > URL 변경) (html 포멧 데이터 수집)
    - 동적페이지 : 페이지의 데이터가 변경될때 URL이 변경되지 않음(더보기 클릭 > URL 그대로에서 내용 변경)(json 포멧 데이터 수집)
        
        정적페이지와 동적 페이지에 따라서 데이터를 수집하는 방법이 다르다
        
- requests package
    - 브라우져의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력 : URL 입력 > Data 받아옴
    - requests 패키지가 이러한 역할을 수행

### Naver Stock Data 수집
- Kospi 지수 데이터
- Kosdaq 지수 데이터
- USD : 원달러 환율 데이터

In [3]:
import requests
import pandas as pd

### 1. 웹서비스를 분석 : 크롬 개발자 도구를 사용 : URL을 활용
### 2. URL을 이용하여 데이터 요청 : requests > response > 최종적으로 json(str)파일을 가져옴
### 3. json(str)파일을 > list나 dict 형으로 받아오고 > DataFrame으로 변환

In [5]:
# 1. 웹서비스 분석
url = 'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=4'

In [6]:
# 2. url 활용 데이터 요청
response = requests.get(url)
response # 여기서 나오는 숫자 코드는 status 코드를 의미

<Response [200]>

In [8]:
#response.text str(문자열) 데이터가 들어가 있는 것을 확인
response.text[:200]

'[{"localTradedAt":"2022-06-22","closePrice":"2,342.81","compareToPreviousClosePrice":"-66.12","compareToPreviousPrice":{"code":"5","text":"하락","name":"FALLING"},"fluctuationsRatio":"-2.74","openPrice"'

In [9]:
#3. 문자열 데이터를 리스트로 변환 > 파일을 보면 리스트 안에 딕셔너리가 있음
data = response.json()
type(data)

list

In [12]:
#3-1 리스트 파일을 데이터프레임으로 변환
# df = pd.DataFrame(data)
df = pd.DataFrame(data)[["localTradedAt", "closePrice"]]
df

,localTradedAt,closePrice
0,2022-06-22,"2,342.81"
1,2022-06-21,"2,408.93"
2,2022-06-20,"2,391.03"
3,2022-06-17,"2,440.93"
4,2022-06-16,"2,451.41"
5,2022-06-15,"2,447.38"
6,2022-06-14,"2,492.97"
7,2022-06-13,"2,504.51"
8,2022-06-10,"2,595.87"
9,2022-06-09,"2,625.44"


In [13]:
# 4 1-3번 함수로 만들어보기 : 파라미터 - pagesize, page

In [16]:
def stock_price(pagesize, page):
    url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={pagesize}&page={page}'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[['localTradedAt', 'closePrice']]

In [18]:
df = stock_price(30,2)
df.tail(2)

,localTradedAt,closePrice
28,2022-05-11,"2,592.27"
29,2022-05-10,"2,596.56"


# Quiz KOSDAQ 데이터 수집 코드 작성

In [22]:
# 1. 웹서비스 분석 : URL

url = 'https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2'

In [23]:
# 2. request(URL) > response(json) : JSON(str)
response = requests.get(url)
response

<Response [200]>

In [28]:
# 3. JSON(str) > list, dict > DataFrame
data = response.json()
df = pd.DataFrame(data)[['localTradedAt','closePrice']]
df.tail(2)

,localTradedAt,closePrice
8,2022-07-08,766.48
9,2022-07-07,757.97


In [30]:
def stock_price(pagesize, page, code="KOSPI"):
    # docstring : 함수를 사용하는 방법을 문자열로 작성 보통 맨 위에 작성
    """ This function is crawling naver stock price from naver webpage
    
    Params
    --------
    pagesize : int : one page size
    page : int : page number
    code : str : choose KOSPI or KOSDAQ
    
    """
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}'
    response = requests.get(url) #네이버 증권 Server에 request 수행하고 Server에서는 DB에 해당하는 내용을 Web Application에서 json으로 만들고 전송
    data = response.json() #데이터를 리스트로 변환
    return pd.DataFrame(data)[['localTradedAt', 'closePrice']] #데이터를 DataFrame으로 변환

In [31]:
# 최근 60일치 데이터 수집

kospi = stock_price(60,1,"KOSPI")
kosdaq = stock_price(60,1,"KOSDAQ")

In [32]:
help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code='KOSPI')
    This function is crawling naver stock price from naver webpage
    
    Params
    --------
    pagesize : int : one page size
    page : int : page number
    code : str : choose KOSPI or KOSDAQ



### 원달러 환율 데이터 수집 연습
- 시장지표 > 미국 USD > 일별시세(더보기)

In [37]:
page, pagesize = 1, 60
url = f'https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page={page}&pageSize={pagesize}'
response = requests.get(url)
data = response.json()
usd = pd.DataFrame(data)[['localTradedAt','closePrice']]
usd.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"1,274.50"
59,2022-05-10,"1,276.00"


In [38]:
kospi.tail(2)

,localTradedAt,closePrice
58,2022-05-11,"2,592.27"
59,2022-05-10,"2,596.56"


In [39]:
kosdaq.tail(2)

,localTradedAt,closePrice
58,2022-05-11,866.34
59,2022-05-10,856.14


In [40]:
# 수집한 데이터로 간단한 데이터 분석 : 상관관계 분석을 해보자
# 상관관계분석 : 두 데이터 집합 사이에 어떤 관계가 있는지 확인하는 분석 방법
# 원달러 환율이 높으면 코스피, 코스닥 지수가 낮다. > 음의 상관관계를 갖는다.

In [41]:
# 피어슨 상관계수를 구하였을때
# 1과 가까울수록 강한 양의 상관관계, -1과 가까울수록 강한 음의 상관관계, 0과 가까울수록 관계가 없다
# 피어슨 상관계수는 DataFrame의 corr 함수를 사용 df.corr()

In [45]:
# 데이터 전처리
df = kospi.copy() # 여기서 copy를 쓴 이유는 밑에서 추가적으로 설명
df['kosdaq'] = kosdaq['closePrice']
df['usd'] = usd['closePrice']
df.rename(columns={"closePrice":'kospi'}, inplace=True)
df

,localTradedAt,kospi,kosdaq,usd
0,2022-08-03,"2,456.72",813.83,"1,308.80"
1,2022-08-02,"2,439.62",804.34,"1,312.00"
2,2022-08-01,"2,452.25",807.61,"1,305.00"
3,2022-07-29,"2,451.50",803.62,"1,304.50"
4,2022-07-28,"2,435.27",798.32,"1,302.00"
5,2022-07-27,"2,415.53",795.70,"1,314.00"
6,2022-07-26,"2,412.96",789.93,"1,311.00"
7,2022-07-25,"2,403.69",789.69,"1,312.00"
8,2022-07-22,"2,393.14",789.75,"1,310.00"
9,2022-07-21,"2,409.16",795.15,"1,315.00"


In [47]:
df[['kospi','kosdaq','usd']].corr() # 바로 하면 오류
df.dtypes # 상관계수를 구해야하는데 변수들이 object형이라 결과가 나오지 않음

localTradedAt    object
kospi            object
kosdaq           object
usd              object
dtype: object

In [49]:
# 변수(컬럼)의 데이터 타입 변경 str > float
# df[column].apply() : 모든 데이터를 괄호 안에 함수에 대입한 결과를 출력
df['kospi'] = df['kospi'].apply(lambda data: float(data.replace(",",""))) #replace는 데이터에서 ,문자를 제거 >> "," 를 ""로 변경
df['kosdaq'] = df['kosdaq'].apply(lambda data: float(data.replace(",","")))
df['usd'] = df['usd'].apply(lambda data: float(data.replace(",","")))

In [50]:
df[['kospi','kosdaq','usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984051,-0.878703
kosdaq,0.984051,1.000000,-0.821785
usd,-0.878703,-0.821785,1.000000


In [51]:
# copy 메소드를 사용하지 않고 대입연산자를 사용하여 리스트를 복사할 경우 얕은 복사가 발생(리스트의 참조 주소값을 대입,복사)
# apply(function) >> 데이터 값을 함수를 적용한 결과로 반환 >> 여러값을 적용할때 for루프 적용 없이 적용할 수 있는 장점

### Summary
- 웹페이지의 종류
    - 정적페이지 : 데이터가 변경될때 URL 변경 O : HTML
    - 동적페이지 : 데이터가 변경될때 URL 변경 X : JSON
    
- 웹크롤링 절차
    1. 웹서비스 분석 : 크롬 개발자 도구 --> URL 찾기
    2. 데이터 요청 : URL을 활용하여 requests > response 값은 무조건 str 형이다
    3. 데이터 변환 : json(str) 포멧을 > list, dict 형변환 > DataFrame으로 변환